In [1]:
import pandas as pd
import json
from urllib.request import urlopen, quote,URLError, HTTPError
import requests
import time

In [2]:
data = pd.read_csv(r"G:\work\logistica\info\shenglist\JD_ZJ_list.csv")

In [3]:
data.columns

Index(['订单编号', '收货地址', '物流单号', '物流签收时间', 'LOGISTICS_COMPANY', '最后送达时间', '所在省',
       '所在市', '所在县', '快递员', '最后一站', '快递员电话', '导航地址', 'cityId', 'countyId',
       'provinceId', 'siteAddress', 'siteBusinessName', 'siteName',
       'siteTelephone', 'cityname', 'siteAddress_all', 'siteAdcode_gaode',
       'siteCity_gaode', 'siteCountry_gaode', 'siteDistrict_gaode',
       'siteFormatted_address_gaode', 'siteLevel_gaode', 'siteLocation_gaode',
       'siteNumber_gaode', 'siteProvince_gaode', 'siteStreet_gaode',
       'siteTownship_gaode', 'site_Lng_gaode', 'site_Lat_gaode'],
      dtype='object')

In [4]:
df = data[[
    "订单编号","导航地址","物流单号","收货地址", 
    'site_Lat_gaode', 'site_Lng_gaode','siteAddress_all','siteName','物流签收时间','最后送达时间','siteLocation_gaode']]

In [5]:
df['导航地址'].isnull().value_counts()



False    5250
Name: 导航地址, dtype: int64

In [6]:
ak='1c2682317206ccaa18ec2487bb45d880'

In [7]:
add = df['导航地址']

In [8]:

def getlatlng_Gaode2(address):
    ak='1c2682317206ccaa18ec2487bb45d880'
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(ak,address)
    data=requests.get(url)
    contest=data.json()
#     contest=contest['geocodes'][0]['location']
    return contest



In [9]:
start_time = time.time()
listA = []
for b in df['导航地址']:
#     print(b)
    dictA = {}
    try:
        temp=getlatlng_Gaode2(b)
    except HTTPError as e:
        print("请求出错")
        pass
    else:    
        if ('geocodes' in temp):
            dictA["re_location_gaode"] = temp['geocodes'][0]['location']
            dictA["re_formatted_address_gaode"] = temp['geocodes'][0]['formatted_address']
            dictA["re_province_gaode"] = temp['geocodes'][0]['province']
            dictA["re_country_gaode"] = temp['geocodes'][0]['country']
            if ('citycode' in temp):
                dictA["re_citycode_gaode"] = temp['geocodes'][0]['citycode']
            else:
                pass
            dictA["re_city_gaode"] = temp['geocodes'][0]['city']
            dictA["re_district_gaode"] = temp['geocodes'][0]['district']
            dictA["re_level_gaode"] = temp['geocodes'][0]['level']
            dictA["re_township_gaode"] = temp['geocodes'][0]['township']
            dictA["re_adcode_gaode"] = temp['geocodes'][0]['adcode']
            dictA["re_street_gaode"] = temp['geocodes'][0]['street']
            dictA["re_number_gaode"] = temp['geocodes'][0]['number']
        else:
            pass
    listA.append(dictA)
#     time.sleep(1)
end_time = time.time()
print("总用时：",end_time-start_time)

总用时： 297.9120397567749


In [10]:
listA[0:3]

[{'re_location_gaode': '120.346846,30.165910',
  're_formatted_address_gaode': '浙江省杭州市萧山区楼下陈村|171号',
  're_province_gaode': '浙江省',
  're_country_gaode': '中国',
  're_city_gaode': '杭州市',
  're_district_gaode': '萧山区',
  're_level_gaode': '门牌号',
  're_township_gaode': [],
  're_adcode_gaode': '330109',
  're_street_gaode': [],
  're_number_gaode': []},
 {'re_location_gaode': '119.944750,30.052360',
  're_formatted_address_gaode': '浙江省杭州市富阳区后拔路|99',
  're_province_gaode': '浙江省',
  're_country_gaode': '中国',
  're_city_gaode': '杭州市',
  're_district_gaode': '富阳区',
  're_level_gaode': '门牌号',
  're_township_gaode': [],
  're_adcode_gaode': '330111',
  're_street_gaode': '后拔路',
  're_number_gaode': '99'},
 {'re_location_gaode': '120.363146,27.686364',
  're_formatted_address_gaode': '浙江省温州市平阳县凤兴东路|5',
  're_province_gaode': '浙江省',
  're_country_gaode': '中国',
  're_city_gaode': '温州市',
  're_district_gaode': '平阳县',
  're_level_gaode': '门牌号',
  're_township_gaode': [],
  're_adcode_gaode': '330326',

In [11]:
df_A = pd.DataFrame(listA)

In [12]:
df_all = pd.concat([df,df_A],axis=1)

In [13]:
df_all.columns

Index(['订单编号', '导航地址', '物流单号', '收货地址', 'site_Lat_gaode', 'site_Lng_gaode',
       'siteAddress_all', 'siteName', '物流签收时间', '最后送达时间', 'siteLocation_gaode',
       're_adcode_gaode', 're_city_gaode', 're_country_gaode',
       're_district_gaode', 're_formatted_address_gaode', 're_level_gaode',
       're_location_gaode', 're_number_gaode', 're_province_gaode',
       're_street_gaode', 're_township_gaode'],
      dtype='object')

In [14]:
df_all["re_location_gaode"][0:10]

0    120.346846,30.165910
1    119.944750,30.052360
2    120.363146,27.686364
3    120.785783,30.731320
4    120.014617,30.204623
5    120.150896,30.171215
6    120.696338,27.746587
7    120.102000,27.792423
8    121.189801,28.750231
9    121.178798,28.793907
Name: re_location_gaode, dtype: object

In [15]:
df_all['re_location_gaode'] = df_all['re_location_gaode'].map(lambda x:str(x))

In [16]:
split = pd.DataFrame((x.split(',') for x in df_all['re_location_gaode']), index=df_all.index, columns=['re_Lng_gaode','re_Lat_gaode'])
df_allA = pd.merge(df_all,split,left_index=True,right_index=True)

In [17]:
print("df_A:{}".format(df_A.shape[0]))
print("df:{}".format(df.shape[0]))
print("df_all:{}".format(df_all.shape[0]))
print("df_allA:{}".format(df_allA.shape[0]))

df_A:5250
df:5250
df_all:5250
df_allA:5250


In [18]:
df_allA.shape

(5250, 24)

In [19]:
outpath = r"G:/work/logistica/info/distance/"
name_UTF8 = "JD_ZJreceiving_lnglat_gaode.csv"
name_ANSI = "JD_ZJreceiving_lnglat_gaode.xlsx"

In [20]:
df_allA.to_excel(outpath + name_ANSI, index=False)
df_allA.to_csv(outpath + name_UTF8, index=False)